# Basic RAG Chunking

## 0. Import Library & Set Environment Variables

In [26]:
import os
from glob import glob
import nest_asyncio
import pandas as pd

from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter, SentenceWindowNodeParser

from langchain.text_splitter import CharacterTextSplitter

from kiwipiepy import Kiwi

from autorag.chunker import Chunker

In [2]:
root_dir = os.getcwd()

In [5]:
nest_asyncio.apply()

In [8]:
result = pd.read_parquet(os.path.join(root_dir, "..", "Parsing", "hybrid_parse", "parsed_result.parquet"))
result

,texts,path,page,last_modified_datetime
0,# 중소벤처기업부 공고 제2024–300호\n\n# 2024년 글로벌 팁스(Glob...,/home/martinus/workspace/RAG-Evaluation/Parsin...,1,2025-04-20
1,# 서류 및 발표평가 결과에 최대 5점까지 가점 부여\n\n|가점 세부 항목|점수|...,/home/martinus/workspace/RAG-Evaluation/Parsin...,10,2025-04-20
2,# 6 유의사항\n\n◈ 창업지원사업에 신청하는 창업기업은 관련 법령에 따라 다음 ...,/home/martinus/workspace/RAG-Evaluation/Parsin...,11,2025-04-20
3,◦ 선정자는 창업진흥원이 지정한 은행 계좌 (사업비계좌) 개설 및 창업 사업통합관리...,/home/martinus/workspace/RAG-Evaluation/Parsin...,15,2025-04-20
4,# 창업여부 기준표\n\n|신청기업구분|상세 구분|창업여부| |\n|---|---|...,/home/martinus/workspace/RAG-Evaluation/Parsin...,17,2025-04-20
5,# 붙임2 지원 제외 대상 업종\n\n▶ 중소기업창업지원법 시행령 제4조(창업...,/home/martinus/workspace/RAG-Evaluation/Parsin...,18,2025-04-20
6,# 붙임3 동시수행 불가한 글로벌 창업지원사업 목록\n\n※ ’24년도 중소벤처기업...,/home/martinus/workspace/RAG-Evaluation/Parsin...,19,2025-04-20
7,# 신산업 창업 분야\n\n※ 「중소기업창업 지원법」 제25조 4항에 따라 신산업 ...,/home/martinus/workspace/RAG-Evaluation/Parsin...,20,2025-04-20
8,# 붙임5 소재·부품·장비 및 스타트업 10대 초격차 분야\n\n☞ 품목별 시장 및...,/home/martinus/workspace/RAG-Evaluation/Parsin...,21,2025-04-20
9,# 전략분야\n\n# 전략품목\n\n|수소 저장·운반용 복합 소재|4D FMCW 라...,/home/martinus/workspace/RAG-Evaluation/Parsin...,22,2025-04-20


In [10]:
texts = result["texts"].tolist()
texts

['# 중소벤처기업부 공고 제2024–300호\n\n# 2024년 글로벌 팁스(Global TIPS) 창업기업 모집 공고\n\n해외투자와 연계하여 유망 창업기업의 해외 진출 및 글로벌 기업으로의 성장을 지원하는 『2024년 글로벌 팁스(Global TIPS)』에 참여할 창업기업을 다음과 같이 모집합니다.\n\n2024년 5월 7일\n\n중소벤처기업부 장관\n\n# 1 사업개요\n\n- 사업목적 : 미래 신성장 동력 발굴과 글로벌 선도기업 육성 등을 위해 해외 벤처캐피탈 등으로부터 투자받은 유망 창업기업의 글로벌 진출 활성화 도모\n- 지원대상 : 해외 VC로부터 20만 불 이상 투자를 유치하고 해외 법인 설립·희망하는 업력 7년 이내 창업기업\n- * 「중소기업창업 지원법」에 따른 신산업 분야([붙임4] 참조) 창업기업인 경우 업력 10년 기업까지 지원 가능\n- 지원내용 :\n|사업화자금 (최대 2억원, 평균 1.8억원)|현지진출 프로그램 등|\n|---|---|\n|시제품 제작, 지재권 취득, 사업모델 (BM) 개선 등에 소요되는 비용 지원|실무교육, 멘토링, 글로벌 네트워킹, 후속 투자유치 등 주관기관의 창업 프로그램 운영|\n- 선정규모 : 20개사 내외\n- 협약기간 : 협약시작일로부터 10개월 이내\n- * 사업기간은 총 3년으로 선정 후, 1년 단위로 협약 연장',
 '# 서류 및 발표평가 결과에 최대 5점까지 가점 부여\n\n|가점 세부 항목|점수|비고|\n|---|---|---|\n|① 팁스 성공 기업 (프리, 시드 포함)|1점| |\n|② 글로벌 지원사업 (글로벌액셀러레이팅, 글로벌창업사관학교, 해외실증 (PoC), KSC 등) 성공(졸업) 기업|1점| |\n|③ 전략 기술분야 (소재‧부품‧장비, 10대 초격차 분야) [붙임5]|1점|최대 5점|\n|④ (해외VC) ‘해외 VC글로벌’ 펀드(한국벤처투자 출자) 투자자|1점| |\n|⑤ (해외VC) 국내 지사·사무소 보유 여부|1점| |\n|⑥ (해외VC) 투자전략 및 지원계획 발표**|1점| 

In [12]:
# cvt into Document instance for LlamaIndex
documents = [Document(text=text) for text in texts]

## 1. Token Text Splitter

### 1-1. Using LlamaIndex

In [14]:
token_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=0)

token_result = token_splitter.get_nodes_from_documents(documents=documents)
token_texts = [x.text for x in token_result]

In [15]:
len(token_texts)

30

## 2. Character Splitter

### 2-1. Using LangChain

> 부정확한 경우가 많음..

In [17]:
char_splitter = CharacterTextSplitter()
char_texts = char_splitter.create_documents(texts=texts)

In [18]:
len(char_texts)

23

## 3. Sentence Splitter

### 3-1. Using Kiwi

In [20]:
kiwi = Kiwi()

kiwi_result = kiwi.split_into_sents(texts[0])
kiwi_sentences = [x.text for x in kiwi_result]

In [21]:
len(kiwi_sentences)

7

## 4. Sentence Window

### 4-1. Using LlamaIndex

In [23]:
window = SentenceWindowNodeParser(window_size=5)

window_result = window.get_nodes_from_documents(documents=documents)
window_texts = [x.text for x in window_result]
window_metadata = [x.metadata for x in window_result]

In [24]:
len(window_texts)

83

In [25]:
window_metadata[0]

{'window': '# 중소벤처기업부 공고 제2024–300호\n\n# 2024년 글로벌 팁스(Global TIPS) 창업기업 모집 공고\n\n해외투자와 연계하여 유망 창업기업의 해외 진출 및 글로벌 기업으로의 성장을 지원하는 『2024년 글로벌 팁스(Global TIPS)』에 참여할 창업기업을 다음과 같이 모집합니다.\n\n 2024년 5월 7일\n\n중소벤처기업부 장관\n\n# 1 사업개요\n\n- 사업목적 : 미래 신성장 동력 발굴과 글로벌 선도기업 육성 등을 위해 해외 벤처캐피탈 등으로부터 투자받은 유망 창업기업의 글로벌 진출 활성화 도모\n- 지원대상 : 해외 VC로부터 20만 불 이상 투자를 유치하고 해외 법인 설립·희망하는 업력 7년 이내 창업기업\n- * 「중소기업창업 지원법」에 따른 신산업 분야([붙임4] 참조) 창업기업인 경우 업력 10년 기업까지 지원 가능\n- 지원내용 :\n|사업화자금 (최대 2억원, 평균 1.8억원)|현지진출 프로그램 등|\n|---|---|\n|시제품 제작, 지재권 취득, 사업모델 (BM) 개선 등에 소요되는 비용 지원|실무교육, 멘토링, 글로벌 네트워킹, 후속 투자유치 등 주관기관의 창업 프로그램 운영|\n- 선정규모 : 20개사 내외\n- 협약기간 : 협약시작일로부터 10개월 이내\n- * 사업기간은 총 3년으로 선정 후, 1년 단위로 협약 연장',
 'original_text': '# 중소벤처기업부 공고 제2024–300호\n\n# 2024년 글로벌 팁스(Global TIPS) 창업기업 모집 공고\n\n해외투자와 연계하여 유망 창업기업의 해외 진출 및 글로벌 기업으로의 성장을 지원하는 『2024년 글로벌 팁스(Global TIPS)』에 참여할 창업기업을 다음과 같이 모집합니다.\n\n'}

## 5. Using AutoRAG

In [27]:
parsed_path = os.path.join(root_dir, "..", "Parsing", "hybrid_parse", "parsed_result.parquet")
pjt_dir = os.path.join(root_dir, "basic_rag_chunking")

chunker = Chunker.from_parquet(parsed_data_path=parsed_path, project_dir=pjt_dir)

chunker.start_chunking(yaml_path=os.path.join(root_dir, "config", "basic_chunking.yaml"))

[04/28/25 14:55:55] INFO     [chunker.py:44] >> Chunking Start...  ]8;id=188871;file:///home/martinus/llm/lib/python3.10/site-packages/autorag/chunker.py\chunker.py]8;;\:]8;id=72245;file:///home/martinus/llm/lib/python3.10/site-packages/autorag/chunker.py#44\44]8;;\
                    INFO     [base.py:22] >> Running chunker -        ]8;id=7113;file:///home/martinus/llm/lib/python3.10/site-packages/autorag/data/chunk/base.py\base.py]8;;\:]8;id=978515;file:///home/martinus/llm/lib/python3.10/site-packages/autorag/data/chunk/base.py#22\22]8;;\
                             llama_index_chunk module...                        
                    INFO     [base.py:22] >> Running chunker -        ]8;id=372658;file:///home/martinus/llm/lib/python3.10/site-packages/autorag/data/chunk/base.py\base.py]8;;\:]8;id=357185;file:///home/martinus/llm/lib/python3.10/site-packages/autorag/data/chunk/base.py#22\22]8;;\
                             llama_index_chunk module...      

## 6. Check Result

In [28]:
token_path = os.path.join(pjt_dir, "0.parquet")
token_result = pd.read_parquet(token_path)

token_result

,doc_id,contents,path,start_end_idx,metadata
0,42816835-008e-4bd6-ac12-064b0f7c10bf,# 중소벤처기업부 공고 제2024–300호\n\n# 2024년 글로벌 팁스(Glob...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 483]","{'last_modified_datetime': '2025-04-20', 'next..."
1,22a32372-9e30-46b5-9187-a05b61e14b76,"등|\n|---|---|\n|시제품 제작, 지재권 취득, 사업모델 (BM) 개선 등...",/home/martinus/workspace/RAG-Evaluation/Parsin...,"[459, 642]","{'last_modified_datetime': '2025-04-20', 'next..."
2,ca206649-a88e-4e7d-a15b-13536888f746,# 서류 및 발표평가 결과에 최대 5점까지 가점 부여\n\n|가점 세부 항목|점수|...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 478]","{'last_modified_datetime': '2025-04-20', 'next..."
3,a17fa2e9-9461-4204-9d6f-7888c1dd0c4d,발표 시 가점 반영\n\n# 사업 운영일정\n\n※ 세부 일정은 대내‧외 사정에 의...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[455, 796]","{'last_modified_datetime': '2025-04-20', 'next..."
4,6e10d94b-fa90-4436-aad7-d0f13eede040,# 6 유의사항\n\n◈ 창업지원사업에 신청하는 창업기업은 관련 법령에 따라 다음 ...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 494]","{'last_modified_datetime': '2025-04-20', 'next..."
5,d4fc6901-849f-48f1-a29f-bbf6badcc91c,"통합관리지침” 제32조, “공공재정 부정청구 금지 및 부정이익 환수 등에 관한 법률...",/home/martinus/workspace/RAG-Evaluation/Parsin...,"[476, 934]","{'last_modified_datetime': '2025-04-20', 'next..."
6,b5a99620-9d72-4263-abe5-c82df2b9395a,◦ 선정자는 창업진흥원이 지정한 은행 계좌 (사업비계좌) 개설 및 창업 사업통합관리...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 458]","{'last_modified_datetime': '2025-04-20', 'next..."
7,bc01335e-b477-40ea-8868-20df58582dd9,및 시스템 문의|중소기업 통합 콜센터|(국번없이) 1357|\n\n프로그램 홈페이지...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[446, 510]","{'last_modified_datetime': '2025-04-20', 'next..."
8,4aa922ae-ce09-43b7-b0fd-59c7c4ca8d69,# 창업여부 기준표\n\n|신청기업구분|상세 구분|창업여부| |\n|---|---|...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 416]","{'last_modified_datetime': '2025-04-20', 'next..."
9,05d36e15-6664-479a-a0ec-8462035e7e8f,계약에 의한 법인전환|개인사업자의 창업자 지위승계| |\n|이종창업|-|창업| |\...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[399, 836]","{'last_modified_datetime': '2025-04-20', 'next..."


In [29]:
window_path = os.path.join(pjt_dir, "1.parquet")
window_result = pd.read_parquet(window_path)

window_result

,doc_id,contents,path,start_end_idx,metadata
0,f64f8384-4be9-465c-ae76-184647bd1e4a,# 중소벤처기업부 공고 제2024–300호,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 22]","{'last_modified_datetime': '2025-04-20', 'next..."
1,4279d5af-fefa-4878-9fcc-022dd09dfeb7,# 2024년 글로벌 팁스(Global TIPS) 창업기업 모집 공고,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[25, 62]","{'last_modified_datetime': '2025-04-20', 'next..."
2,6522a4d5-034c-4eef-9b44-b9a03d142763,해외투자와 연계하여 유망 창업기업의 해외 진출 및 글로벌 기업으로의 성장을 지원하는...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[65, 163]","{'last_modified_datetime': '2025-04-20', 'next..."
3,23c1375f-3358-461d-9c00-53df4ce3e1cb,2024년 5월 7일,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[166, 176]","{'last_modified_datetime': '2025-04-20', 'next..."
4,5e88bbd2-0c57-48fd-aceb-6404c1498ff3,중소벤처기업부 장관,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[179, 188]","{'last_modified_datetime': '2025-04-20', 'next..."
...,...,...,...,...,...
176,0b447342-f615-4bd6-8d43-3ee0d319b38b,4 신청 및 접수\n□ 신청 ‧ 접수 기간\n◦ 접수기간 : 2024. 5. 7.(...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 105]","{'last_modified_datetime': '2025-04-20', 'next..."
177,4688e5dd-d88b-4673-a1f3-b01bc8a430dd,"신청 마감일에는 문의 및 접속이 원활하지 않을 수 있으므로, 마감일 2~3일 이전에...",/home/martinus/workspace/RAG-Evaluation/Parsin...,"[107, 259]","{'last_modified_datetime': '2025-04-20', 'next..."
178,d4f1d516-bb1d-4211-82e3-0e653336cc1c,"다만, 16:00 전에 1단계(약관동의) 후\n신청서 작성 단계에 진입하여 과제번호...",/home/martinus/workspace/RAG-Evaluation/Parsin...,"[261, 384]","{'last_modified_datetime': '2025-04-20', 'next..."
179,2069f832-93a8-4634-8bce-cd92870ae4b2,"온라인으로만 신청이 가능하고, 신청 마감일 18시 이후에는 사업계획서 등 일체\n내...",/home/martinus/workspace/RAG-Evaluation/Parsin...,"[386, 917]","{'last_modified_datetime': '2025-04-20', 'next..."
